# ChessML Netz

This neural network based on PyTorch will try to classifiy chess pieces from photos taken from the top of a chessboard.

In [1]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import progressbar

from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image

## Define a normalization function for the analyzed data

The values are recommended by a PyTorch tutorial (https://youtu.be/32lHVbT09h8).

In [2]:
train_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

## Reading the data

First defining a sampler for development, so that we don't have to load all the data everytime we run the net.

In [3]:
#Training
n_training_samples = 1000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 20
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 4
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))


In [4]:
# Train Data
train_set = torchvision.datasets.ImageFolder(root="./data/binary/train",
                                             transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=2,
                                           #sampler=train_sampler
                                           )

# Validation Data
val_set = torchvision.datasets.ImageFolder(root="./data/binary/validation",
                                           transform=train_transform)
val_loader = torch.utils.data.DataLoader(val_set,
                                         batch_size=4,
                                         shuffle=True,
                                         num_workers=2,
                                         #sampler=val_sampler
                                         )

test_data = torchvision.datasets.ImageFolder("./data/validation",
                                             transform=train_transform)
test_loader = torch.utils.data.DataLoader(val_set,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=2,
                                          #sampler=val_sampler
                                          )


## Defining classes

bb = Black Bishop
bk = Black King
bn = Black Knight
bp = Black Pawn
bq = Black Queen
br = Black Rook
...

In [5]:
classes = ("bb", "bk", "bn", "bp", "bq", "br","empty", "wb", "wk", "wn", "wp", "wq", "wr")

## Defining the Neural Network

In [6]:
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        # Defining the convolutional layers of the net
        self.conv1 = nn.Conv2d(3, 12, kernel_size=5)
        self.conv2 = nn.Conv2d(12, 24, kernel_size=5)
        
        # self.dropout = nn.Dropout()

        # Defining the fully connected layers of the net
        self.fc1 = nn.Linear(600, 64)  
        self.fc2 = nn.Linear(64, 13)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)  # Relu because it's famous

        x = self.conv2(x)
        # x = self.dropout(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = x.view(-1, 600)  # Convert 2d data to 1d

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)

        return x


## Training

In [7]:
def train(model, epoch, optimizer, criterion):
    model.train()
    running_loss = 0.0
    with progressbar.ProgressBar(max_value=len(train_loader)) as bar:
        for i, t_data in enumerate(train_loader):
            data, target = t_data
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            out = model(data)
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            bar.update(i)
            if i % 2000 == 1999:
                print("Loss:", running_loss / 2000)
                running_loss = 0.0

## Validation

In [8]:
def validate(model, criterion, epoch=0):
    model.eval()
    correct = 0
    total = 0
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))
    predictions_for_class = [[0] for i in range(len(classes))] 
    with torch.no_grad():
        for data, target in val_loader:
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            out = model(data)
            _, prediction = torch.max(out.data, 1)
            total += target.size(0)
            if torch.cuda.is_available():
                correct += prediction.eq(target).sum().cpu().item()
            else:
                correct += prediction.eq(target).sum().item()
                
            c = (prediction == target).squeeze()
            for i in range(target.size(0)):
                label = target[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    print("\nValidation")
    print("###################################")
    print("Epoch", epoch)
    print("Accuracy: %f" % (100 * correct / total))
    print("###################################\n")
    for i in range(len(classes)):
        try:
            print('Accuracy of %5s : %2d %% [%2d/%2d]' % 
                  (classes[i], 100 * class_correct[i] / class_total[i], class_correct[i], class_total[i]))
        except ZeroDivisionError:
            print('No Accuracy for %s' % classes[i])
            


## Testing

In [9]:
def test(model):
    model.eval()
    
    dataiter = iter(test_loader)
    images, labels = dataiter.next()
     
    out = model(images)
    
    _, predicted = torch.max(out, 1)

    imshow(torchvision.utils.make_grid(images))
    print('   Actual: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))


## Helper functions


In [10]:
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


The save model function will save the state of a model after a specific epoch. 

In [11]:
def save_model(model, epoch):
    torch.save(model.state_dict(), "model/chess-net_{}.pt".format(epoch))
    print("\n------- Checkpoint saved -------\n")

## Main

Training and evaluating the ChessNet

In [12]:
def main():
    model = ChessNet()
    
    # Activate cuda support if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Check if model is already available
    # if os.path.isfile("model/chess-net.pt"):
    #    model = torch.load("model/chess-net.pt")
    
    # Defining the loss function
    criterion = nn.CrossEntropyLoss()

    # Defining the optimizer
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Start training
    epochs = 20
    start = time.time()
    print("Starting training for %s epochs on %s" % (epochs, time.ctime()))
    for epoch in range(epochs):
        train(model, epoch, optimizer, criterion)
        validate(model, criterion, epoch)
        save_model(model, epoch)
    end = time.time()
    print("Training of the neuroal network done.")
    print("Time spent:", end-start, "s")
    
    # Testing the NN
    print("\nTest:")
    for i in range(4):
        test(model)

In [13]:
if __name__ == "__main__":
    main()


Starting training for 20 epochs on Fri Mar  8 18:41:28 2019


100% (10348 of 10348) |##################| Elapsed Time: 0:00:12 Time:  0:00:12
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-972361fa1b80>", line 2, in <module>
    main()
  File "<ipython-input-12-05dfe7c9820f>", line 23, in main
    train(model, epoch, optimizer, criterion)
  File "<ipython-input-7-2325408d5c87>", line 17, in train
    loss.backward()
  File "/Users/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/torch/tensor.py", line 102, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/Users/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/torch/autograd/__init__.py", line 90, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michaelw

KeyboardInterrupt: 